## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.90,79,98,4.92,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,73.09,73,6,19.01,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,44.71,53,75,24.16,light rain
3,3,New Norfolk,AU,-42.7826,147.0587,55.76,81,100,1.92,light rain
4,4,Mayumba,GA,-3.4320,10.6554,76.46,84,98,7.78,overcast clouds


In [37]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [38]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,73.09,73,6,19.01,clear sky
4,4,Mayumba,GA,-3.4320,10.6554,76.46,84,98,7.78,overcast clouds
5,5,San Patricio,US,28.0170,-97.5169,78.89,79,100,9.08,moderate rain
9,9,Puerto Ayora,EC,-0.7393,-90.3518,78.76,82,88,10.40,overcast clouds
10,10,Namibe,AO,-15.1961,12.1522,64.69,74,16,12.68,few clouds
11,11,East London,ZA,-33.0153,27.9116,63.54,94,75,16.11,light rain
14,14,Half Moon Bay,US,37.4636,-122.4286,80.92,64,1,0.00,clear sky
15,15,Faanui,PF,-16.4833,-151.7500,79.12,70,74,18.23,broken clouds
17,17,Avarua,CK,-21.2078,-159.7750,75.25,83,93,9.22,overcast clouds
18,18,Melilla,ES,35.2937,-2.9383,77.20,73,20,4.61,few clouds


In [39]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,73.09,73,6,19.01,clear sky
4,4,Mayumba,GA,-3.4320,10.6554,76.46,84,98,7.78,overcast clouds
5,5,San Patricio,US,28.0170,-97.5169,78.89,79,100,9.08,moderate rain
9,9,Puerto Ayora,EC,-0.7393,-90.3518,78.76,82,88,10.40,overcast clouds
10,10,Namibe,AO,-15.1961,12.1522,64.69,74,16,12.68,few clouds
11,11,East London,ZA,-33.0153,27.9116,63.54,94,75,16.11,light rain
14,14,Half Moon Bay,US,37.4636,-122.4286,80.92,64,1,0.00,clear sky
15,15,Faanui,PF,-16.4833,-151.7500,79.12,70,74,18.23,broken clouds
17,17,Avarua,CK,-21.2078,-159.7750,75.25,83,93,9.22,overcast clouds
18,18,Melilla,ES,35.2937,-2.9383,77.20,73,20,4.61,few clouds


In [41]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,73.09,clear sky,-23.1203,-134.9692,
4,Mayumba,GA,76.46,overcast clouds,-3.4320,10.6554,
5,San Patricio,US,78.89,moderate rain,28.0170,-97.5169,
9,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
10,Namibe,AO,64.69,few clouds,-15.1961,12.1522,
11,East London,ZA,63.54,light rain,-33.0153,27.9116,
14,Half Moon Bay,US,80.92,clear sky,37.4636,-122.4286,
15,Faanui,PF,79.12,broken clouds,-16.4833,-151.7500,
17,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,
18,Melilla,ES,77.20,few clouds,35.2937,-2.9383,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
    
    
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,73.09,clear sky,-23.1203,-134.9692,People ThankYou
4,Mayumba,GA,76.46,overcast clouds,-3.4320,10.6554,LIKWALE LODGE
5,San Patricio,US,78.89,moderate rain,28.0170,-97.5169,"Motel 6 Sinton, TX"
9,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Namibe,AO,64.69,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
11,East London,ZA,63.54,light rain,-33.0153,27.9116,Tu Casa
14,Half Moon Bay,US,80.92,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
15,Faanui,PF,79.12,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,Paradise Inn
18,Melilla,ES,77.20,few clouds,35.2937,-2.9383,Hotel Nacional


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))